In [4]:
import sounddevice as sd
import numpy as np
import json
from scipy.signal import resample
from vosk import Model, KaldiRecognizer

MOT_ATTENDU = "bonjour"

MIC_DEVICE = 0

# Charger le modèle Vosk
model = Model("vosk-model-small-fr-0.22")
VOSK_RATE = 16000
recognizer = KaldiRecognizer(model, VOSK_RATE)

# On laisse sounddevice choisir le sample rate natif
device_info = sd.query_devices(MIC_DEVICE, 'input')
MIC_RATE = int(device_info['default_samplerate'])
BLOCKSIZE = 32000

print(f"Micro détecté : {device_info['name']}, rate : {MIC_RATE}")

def callback(indata, frames, time, status):
    if status:
        print(status)

    # convertir en mono
    audio = indata[:, 0]

    # Resample ici pour Vosk
    num_samples = int(len(audio) * VOSK_RATE / MIC_RATE)
    audio_resampled = resample(audio, num_samples)

    audio_bytes = (audio_resampled * 32767).astype(np.int16).tobytes()

    if recognizer.AcceptWaveform(audio_bytes):
        texte = json.loads(recognizer.Result()).get("text", "")
        if texte:
            print("Reconnu :", texte)
            if MOT_ATTENDU in texte:
                print("TRUE")
            else:
                print("FALSE")

with sd.InputStream(
    device=MIC_DEVICE,
    channels=1,
    samplerate=MIC_RATE,
    dtype='float32',
    blocksize=BLOCKSIZE,
    callback=callback
):
    print("Dis le mot attendu")
    while True:
        sd.sleep(1000)


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=4
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-fr-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-fr-0.22/graph/HCLr.fst vosk-model-small-fr-0.22/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo vosk-model-small-fr-0.22/graph/phones/word_boundary.int


Micro détecté : HDA Intel PCH: CX8200 Analog (hw:0,0), rate : 48000
Dis le mot attendu
input overflow
input overflow
input overflow
input overflow
Reconnu : bonjour
TRUE
input overflow
input overflow
Reconnu : voir
FALSE
input overflow


KeyboardInterrupt: 